In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q anthropic pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.1 MB/s eta 0:00:00


In [3]:
# Test API key in Kaggle
from kaggle_secrets import UserSecretsClient
import anthropic

# Get API key from secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("ANTHROPIC_API_KEY")

if api_key:
    print(f"✅ API key loaded: {api_key[:20]}...")
else:
    print("❌ API key not found!")
    
# Test API call
client = anthropic.Anthropic(api_key=api_key)

print("🤖 Testing Claude API from Kaggle...")

message = client.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=30,
    messages=[{
        "role": "user",
        "content": "Say 'Kaggle test successful!'"
    }]
)

print(f"📥 Response: {message.content[0].text}")
print(f"📊 Tokens: {message.usage.input_tokens} in, {message.usage.output_tokens} out")

cost = (message.usage.input_tokens * 3 / 1_000_000) + (message.usage.output_tokens * 15 / 1_000_000)
print(f"💰 Cost: ${cost:.6f}")

print("\n✅ Everything works in Kaggle!")


✅ API key loaded: sk-ant-api03-uI4Nl0Y...
🤖 Testing Claude API from Kaggle...
📥 Response: Kaggle test successful!
📊 Tokens: 16 in, 9 out
💰 Cost: $0.000183

✅ Everything works in Kaggle!


In [4]:
# Cell 1: Download Chrysler Credit Agreement
import urllib.request
import os

print("="*60)
print("  DOWNLOADING TALF LLC CREDIT AGREEMENT")
print("  U.S. Treasury Public Document (2009)")
print("="*60 + "\n")

# Download from Treasury website
url = "https://home.treasury.gov/system/files/136/SPV-Credit-Agt.pdf"
pdf_path = "/kaggle/working/chrysler_docs.pdf"

print("📥 Downloading from Treasury.gov...")
urllib.request.urlretrieve(url, pdf_path)

# Check file
file_size = os.path.getsize(pdf_path) / (1024 * 1024)  # Convert to MB
print(f"✅ Downloaded: {file_size:.2f} MB")
print(f"📁 Location: {pdf_path}\n")


  DOWNLOADING TALF LLC CREDIT AGREEMENT
  U.S. Treasury Public Document (2009)

📥 Downloading from Treasury.gov...
✅ Downloaded: 0.19 MB
📁 Location: /kaggle/working/chrysler_docs.pdf



In [5]:
# Cell 2: PDF Text Extractor
import pdfplumber

def extract_pdf_text(pdf_path, max_pages=None):
    """
    Extract text from PDF
    
    Args:
        pdf_path: Path to PDF file
        max_pages: Optional limit on pages to process
        
    Returns:
        Dictionary with extracted text and metadata
    """
    print(f"📄 Extracting text from PDF...")
    print(f"   File: {pdf_path}\n")
    
    result = {
        'text': '',
        'pages_processed': 0,
        'total_pages': 0,
        'tables_found': 0
    }
    
    with pdfplumber.open(pdf_path) as pdf:
        result['total_pages'] = len(pdf.pages)
        pages_to_process = pdf.pages[:max_pages] if max_pages else pdf.pages
        
        print(f"   Total pages in document: {result['total_pages']}")
        print(f"   Processing: {len(pages_to_process)} pages\n")
        
        for i, page in enumerate(pages_to_process, 1):
            # Extract text
            page_text = page.extract_text()
            if page_text:
                result['text'] += page_text + "\n\n"
            
            # Count tables
            tables = page.extract_tables()
            if tables:
                result['tables_found'] += len(tables)
            
            # Progress update every 10 pages
            if i % 10 == 0:
                print(f"   ✓ Processed {i}/{len(pages_to_process)} pages...")
        
        result['pages_processed'] = len(pages_to_process)
        
        print(f"\n✅ Extraction complete!")
        print(f"   Pages processed: {result['pages_processed']}")
        print(f"   Characters extracted: {len(result['text']):,}")
        print(f"   Tables found: {result['tables_found']}")
        print(f"   Estimated tokens: ~{len(result['text']) // 4:,}\n")
    
    return result

# Test the extractor
print("🧪 Testing PDF extractor...\n")
extraction = extract_pdf_text(pdf_path, max_pages=54)  # First 30 pages

print(f"📝 First 500 characters of extracted text:")
print("-" * 60)
print(extraction['text'][:500])
print("...")
print("-" * 60)


🧪 Testing PDF extractor...

📄 Extracting text from PDF...
   File: /kaggle/working/chrysler_docs.pdf

   Total pages in document: 54
   Processing: 54 pages

   ✓ Processed 10/54 pages...
   ✓ Processed 20/54 pages...
   ✓ Processed 30/54 pages...
   ✓ Processed 40/54 pages...
   ✓ Processed 50/54 pages...

✅ Extraction complete!
   Pages processed: 54
   Characters extracted: 112,846
   Tables found: 1
   Estimated tokens: ~28,211

📝 First 500 characters of extracted text:
------------------------------------------------------------
EXECUTION COPY
CREDIT AGREEMENT
among
TALF LLC,
as Borrower,
FEDERAL RESERVE BANK OF NEW YORK,
as Controlling Party,
FEDERAL RESERVE BANK OF NEW YORK,
as the Senior Lender
and
UNITED STATES DEPARTMENT OF THE TREASURY,
as the Subordinated Lender
Dated as of March 3, 2009
(NY) 07865/007/LLC/SPV.Credit.Agreement.doc

TABLE OF CONTENTS
PAGE
ARTICLE 1
DEFINITIONS
Section 1.01. Defined Terms........................................................................

In [6]:
# Cell 3: Claude Credit Term Extractor
from kaggle_secrets import UserSecretsClient
import anthropic
import json
import re

class CreditTermExtractor:
    """Extract structured credit terms using Claude API"""
    
    def __init__(self):
        # Get API key from Kaggle secrets
        user_secrets = UserSecretsClient()
        self.api_key = user_secrets.get_secret("ANTHROPIC_API_KEY")
        
        if not self.api_key:
            raise ValueError("❌ API key not found in Kaggle secrets!")
        
        self.client = anthropic.Anthropic(api_key=self.api_key)
        self.model = "claude-sonnet-4-5-20250929"
        
        print("✅ Credit Term Extractor initialized")
        print(f"   Model: {self.model}\n")
    
    def extract(self, document_text, max_chars=100000):
        """
        Extract credit terms from document text
        
        Args:
            document_text: Full text extracted from PDF
            max_chars: Maximum characters to send (to stay within token limits)
            
        Returns:
            Dictionary with extracted credit terms
        """
        print("🤖 Analyzing document with Claude API...\n")
        
        # Truncate if needed
        if len(document_text) > max_chars:
            print(f"⚠️  Document truncated from {len(document_text):,} to {max_chars:,} chars")
            document_text = document_text[:max_chars]
        
        # Build prompt
        prompt = self._build_extraction_prompt(document_text)
        
        # Call Claude API
        try:
            message = self.client.messages.create(
                model=self.model,
                max_tokens=4096,
                temperature=0,  # Deterministic for extraction
                messages=[{
                    "role": "user",
                    "content": prompt
                }]
            )
            
            response_text = message.content[0].text
            
            # Show API usage
            print(f"📊 API Usage:")
            print(f"   Input tokens: {message.usage.input_tokens:,}")
            print(f"   Output tokens: {message.usage.output_tokens:,}")
            
            # Calculate cost
            input_cost = message.usage.input_tokens * 3 / 1_000_000
            output_cost = message.usage.output_tokens * 15 / 1_000_000
            total_cost = input_cost + output_cost
            
            print(f"   Total cost: ${total_cost:.4f}\n")
            
            # Parse JSON response
            extracted_data = self._parse_json_response(response_text)
            
            return extracted_data
            
        except Exception as e:
            print(f"❌ API Error: {e}")
            return {"error": str(e)}
    
    def _build_extraction_prompt(self, document_text):
        """Build the extraction prompt for Claude"""
        
        prompt = f"""You are an expert financial analyst specializing in credit agreements. Analyze this credit document and extract key terms.

DOCUMENT TEXT:
{document_text}

Extract the following information and return ONLY valid JSON (no markdown, no code blocks, no explanation):

{{
    "document_info": {{
        "document_type": "Type of document",
        "date": "Document date",
        "parties_involved": ["List of all parties"]
    }},
    "borrower": {{
        "name": "Full legal name",
        "entity_type": "Corporation/LLC/etc",
        "jurisdiction": "State/Country"
    }},
    "lender": {{
        "name": "Lender name",
        "type": "Bank/Government/Private"
    }},
    "loan_details": {{
        "total_amount": "Total loan amount with currency",
        "facility_type": "Type of credit facility",
        "purpose": "Purpose of the loan"
    }},
    "interest_terms": {{
        "base_rate": "Base interest rate or reference rate",
        "margin": "Spread/margin over base rate",
        "total_rate": "All-in interest rate",
        "payment_frequency": "How often interest is paid"
    }},
    "maturity": {{
        "effective_date": "Start date (YYYY-MM-DD)",
        "maturity_date": "End date (YYYY-MM-DD)",
        "term": "Length of loan"
    }},
    "fees": {{
        "origination_fee": "Upfront fee",
        "commitment_fee": "Ongoing commitment fee",
        "other_fees": ["List of other fees"]
    }},
    "financial_covenants": [
        {{
            "covenant_name": "Name of covenant",
            "threshold": "Required ratio/amount",
            "testing_frequency": "When tested"
        }}
    ],
    "collateral": [
        {{
            "type": "Type of collateral",
            "description": "Description",
            "priority": "First lien/Second lien/etc"
        }}
    ],
    "guarantees": {{
        "government_guarantee": "Yes/No and details",
        "guarantors": ["List of guarantors"]
    }},
    "key_conditions": [
        "List key conditions, representations, and covenants"
    ],
    "default_provisions": [
        "Key events of default"
    ],
    "prepayment_terms": {{
        "allowed": true/false,
        "conditions": "Conditions for prepayment",
        "penalties": "Any prepayment penalties"
    }},
    "special_provisions": [
        "Any unique or special terms (like government involvement, bailout terms, etc.)"
    ]
}}

If any information is not found in the document, use null.
Be thorough and accurate. Extract all key financial and legal terms."""

        return prompt
    
    def _parse_json_response(self, response_text):
        """Extract and parse JSON from Claude's response"""
        
        # Try to find JSON in the response
        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
        
        if json_match:
            try:
                data = json.loads(json_match.group())
                print("✅ Successfully parsed JSON response\n")
                return data
            except json.JSONDecodeError as e:
                print(f"⚠️  JSON parse error: {e}")
                return {
                    "error": "Could not parse JSON",
                    "raw_response": response_text[:1000]
                }
        else:
            print("⚠️  No JSON found in response")
            return {
                "error": "No JSON in response",
                "raw_response": response_text[:1000]
            }

# Initialize the extractor
extractor = CreditTermExtractor()


✅ Credit Term Extractor initialized
   Model: claude-sonnet-4-5-20250929



In [7]:
# Cell 4: Complete Extraction Pipeline
print("="*70)
print("  TALF LLC CREDIT AGREEMENT ANALYSIS")
print("  Extracting Terms with AI")
print("="*70 + "\n")

# Step 1: Extract credit terms
credit_terms = extractor.extract(extraction['text'])

# Step 2: Display results
print("="*70)
print("  EXTRACTED CREDIT TERMS")
print("="*70 + "\n")

# Pretty print the JSON
print(json.dumps(credit_terms, indent=2))

print("\n" + "="*70)
print("  EXTRACTION SUMMARY")
print("="*70)
print(f"✅ Document analyzed successfully")
print(f"📄 Pages processed: {extraction['pages_processed']}")
print(f"📊 Data fields extracted: {len(credit_terms)}")
print("="*70)


  TALF LLC CREDIT AGREEMENT ANALYSIS
  Extracting Terms with AI

🤖 Analyzing document with Claude API...

⚠️  Document truncated from 112,846 to 100,000 chars
📊 API Usage:
   Input tokens: 24,538
   Output tokens: 1,660
   Total cost: $0.0985

✅ Successfully parsed JSON response

  EXTRACTED CREDIT TERMS

{
  "document_info": {
    "document_type": "Credit Agreement",
    "date": "2009-03-03",
    "parties_involved": [
      "TALF LLC (Borrower)",
      "Federal Reserve Bank of New York (Controlling Party)",
      "Federal Reserve Bank of New York (Senior Lender)",
      "United States Department of the Treasury (Subordinated Lender)"
    ]
  },
  "borrower": {
    "name": "TALF LLC",
    "entity_type": "Limited Liability Company",
    "jurisdiction": "Delaware"
  },
  "lender": {
    "name": "Federal Reserve Bank of New York (Senior Lender) and United States Department of the Treasury (Subordinated Lender)",
    "type": "Government"
  },
  "loan_details": {
    "total_amount": "$200,0

In [8]:
# Cell 5: Create Visual Summary
def print_credit_summary(terms):
    """Print a formatted summary of credit terms"""
    
    print("\n" + "="*70)
    print("  📋 CREDIT AGREEMENT SUMMARY")
    print("="*70 + "\n")
    
    # Borrower & Lender
    if 'borrower' in terms and terms['borrower']:
        print("👤 BORROWER:")
        print(f"   {terms['borrower'].get('name', 'N/A')}")
        print()
    
    if 'lender' in terms and terms['lender']:
        print("🏦 LENDER:")
        print(f"   {terms['lender'].get('name', 'N/A')}")
        print(f"   Type: {terms['lender'].get('type', 'N/A')}")
        print()
    
    # Loan Amount
    if 'loan_details' in terms and terms['loan_details']:
        print("💰 LOAN DETAILS:")
        print(f"   Amount: {terms['loan_details'].get('total_amount', 'N/A')}")
        print(f"   Type: {terms['loan_details'].get('facility_type', 'N/A')}")
        print(f"   Purpose: {terms['loan_details'].get('purpose', 'N/A')}")
        print()
    
    # Interest Rate
    if 'interest_terms' in terms and terms['interest_terms']:
        print("📈 INTEREST TERMS:")
        int_terms = terms['interest_terms']
        print(f"   Base Rate: {int_terms.get('base_rate', 'N/A')}")
        print(f"   Margin: {int_terms.get('margin', 'N/A')}")
        print(f"   Total Rate: {int_terms.get('total_rate', 'N/A')}")
        print()
    
    # Maturity
    if 'maturity' in terms and terms['maturity']:
        print("📅 TERM:")
        mat = terms['maturity']
        print(f"   Effective: {mat.get('effective_date', 'N/A')}")
        print(f"   Maturity: {mat.get('maturity_date', 'N/A')}")
        print(f"   Duration: {mat.get('term', 'N/A')}")
        print()
    
    # Covenants
    if 'financial_covenants' in terms and terms['financial_covenants']:
        print("📊 FINANCIAL COVENANTS:")
        for i, cov in enumerate(terms['financial_covenants'][:5], 1):
            print(f"   {i}. {cov.get('covenant_name', 'N/A')}: {cov.get('threshold', 'N/A')}")
        if len(terms['financial_covenants']) > 5:
            print(f"   ... and {len(terms['financial_covenants']) - 5} more")
        print()
    
    # Collateral
    if 'collateral' in terms and terms['collateral']:
        print("🏛️  COLLATERAL:")
        for i, col in enumerate(terms['collateral'][:5], 1):
            if isinstance(col, dict):
                print(f"   {i}. {col.get('type', 'N/A')}")
            else:
                print(f"   {i}. {col}")
        if len(terms['collateral']) > 5:
            print(f"   ... and {len(terms['collateral']) - 5} more")
        print()
    
    # Special Provisions
    if 'special_provisions' in terms and terms['special_provisions']:
        print("⭐ SPECIAL PROVISIONS:")
        for i, prov in enumerate(terms['special_provisions'][:3], 1):
            print(f"   {i}. {prov}")
        if len(terms['special_provisions']) > 3:
            print(f"   ... and {len(terms['special_provisions']) - 3} more")
        print()
    
    print("="*70 + "\n")

# Generate summary
print_credit_summary(credit_terms)


  📋 CREDIT AGREEMENT SUMMARY

👤 BORROWER:
   TALF LLC

🏦 LENDER:
   Federal Reserve Bank of New York (Senior Lender) and United States Department of the Treasury (Subordinated Lender)
   Type: Government

💰 LOAN DETAILS:
   Amount: $200,000,000,000 USD (Senior: $180,000,000,000; Subordinated: $20,000,000,000)
   Type: Term Loan Facility with Senior and Subordinated Tranches
   Purpose: Financing the acquisition of assets pursuant to the Put Option Agreement in connection with the Term Asset-Backed Securities Loan Facility (TALF)

📈 INTEREST TERMS:
   Base Rate: LIBOR (one-month deposits in Dollars)
   Margin: Senior Loans: LIBOR + 1.0%; Subordinated Loans: LIBOR + 3.0%
   Total Rate: Senior: LIBOR + 1.0%; Subordinated: LIBOR + 3.0%

📅 TERM:
   Effective: 2009-03-03
   Maturity: 2019-03-03
   Duration: 10 years (with extension provisions subject to Controlling Party and Subordinated Lender consent)

📊 FINANCIAL COVENANTS:
   1. Financial Statements - Annual Audited: Within 120 days aft

In [9]:
# Cell 6: Save Extracted Data
import json
from datetime import datetime

# Prepare output
output_data = {
    'metadata': {
        'source_document': 'TALF LLC Credit Agreement (U.S. Treasury)',
        'source_url': 'https://home.treasury.gov/system/files/136/SPV-Credit-Agt.pdf',
        'extraction_date': datetime.now().isoformat(),
        'pages_analyzed': extraction['pages_processed'],
        'model_used': 'claude-sonnet-4-5-20250929'
    },
    'extracted_terms': credit_terms
}

# Save to file
output_file = '/kaggle/working/TALF_LLC_extracted_terms.json'
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=2)

print(f"💾 Extracted data saved to: {output_file}")
print(f"📊 File size: {os.path.getsize(output_file) / 1024:.2f} KB")

# You can download this file from Kaggle's output section
print("\n📥 To download: Check the right sidebar → Output → Download")


💾 Extracted data saved to: /kaggle/working/TALF_LLC_extracted_terms.json
📊 File size: 6.46 KB

📥 To download: Check the right sidebar → Output → Download
